<div id="top">
<!-- <div style="background-image: url(https://www.emtmadrid.es/getattachment/da3be644-cb9d-44db-8011-e3f40f1c5c34); opacity: 0.2"/> -->
<img src="https://www.gmv.com/sites/default/files/content/image/2021/11/03/115/gmv_rgbredblack.png" alt="GMV Logo" style="width: 200px">
<img src="https://www.upm.es/sfs/Rectorado/Gabinete%20del%20Rector/Logos/UPM/CEI/LOGOTIPO%20leyenda%20color%20JPG%20p.png" alt="UPM Logo" style="float: right; width: 200px">
<h1><b>QA: Informo dataset 🚦</b></h1>
<h5 style="text-align: right">INESDATA-MOV</h5>
</div>

# Análisis de calidad
Este cuaderno analiza la calidad del dataset proveniente de la fuente de datos de Información de Movilidad de Madrid ([Informo](https://informo.madrid.es/informo/tmadrid)). La calidad del mismo se validará teniendo en cuenta los siguientes aspectos:
* Análisis de las variables
* Conversiones de tipos de datos
* Checks de calidad del dato
* Análisis Exploratorio de los datos (EDA)

La **calidad del dato** se refiere a la medida en que los datos son adecuados para su uso, por lo que es esencial para garantizar la confiabilidad y utilidad de los datos en diversas aplicaciones y contextos. Así, en este notebook se evaluarán también las cinco dimensiones de la calidad del dato:
1. **Unicidad**: Ausencia de duplicados o registros repetidos en un conjunto de datos. Los datos son únicos cuando cada registro o entidad en el conjunto de datos es único y no hay duplicados presentes.
2. **Exactitud**: Los datos exactos son libres de errores y representan con precisión la realidad que están destinados a describir. Esto implica que los datos deben ser correctos y confiables para su uso en análisis y toma de decisiones.
3. **Completitud**: Los datos completos contienen toda la información necesaria para el análisis y no tienen valores faltantes o nulos que puedan afectar la interpretación o validez de los resultados.
4. **Consistencia**: Los datos consistentes mantienen el mismo formato, estructura y significado en todas las instancias, lo que facilita su comparación y análisis sin ambigüedad.
5. **Validez**: Medida en que los datos son precisos y representan con exactitud la realidad que están destinados a describir. 

<div class="admonition info">
<p class="admonition-title">Nota</p>
<p>
Este dataset ha sido creado ejecutando el comando <code>create</code> del paquete de Python <a href="https://github.com/oeg-upm/inesdata-mov-data-generation"><code>inesdata_mov_datasets</code></a>.<br>
Para poder ejecutar este comando es necesario haber ejecutado antes el comando <code>extract</code>, que realiza la extracción de datos de la API de Informo y los almacena en Minio. El comando <code>create</code> se encargaría de descargar dichos datos y unirlos todos en un único dataset.
</p>
</div>

In [1]:
import os
from datetime import datetime

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from ydata_profiling import ProfileReport

sns.set_palette("deep")
import warnings

warnings.filterwarnings("ignore")

In [2]:
ROOT_PATH = os.path.dirname(os.path.dirname(os.path.abspath(os.getcwd())))
DATA_PATH = os.path.join(ROOT_PATH, "data", "processed")
INFORMO_DATA_PATH = os.path.join(DATA_PATH, "informo")
INFORMO_DATA_PATH

'/home/code/inesdata-mov/data-generation/data/processed/informo'

In [3]:
for w in os.walk(INFORMO_DATA_PATH):
    print(w)

('/home/code/inesdata-mov/data-generation/data/processed/informo', ['2024'], [])
('/home/code/inesdata-mov/data-generation/data/processed/informo/2024', ['03'], [])
('/home/code/inesdata-mov/data-generation/data/processed/informo/2024/03', ['06', '07', '12', '02', '05', '11', '04', '03', '08', '13'], [])
('/home/code/inesdata-mov/data-generation/data/processed/informo/2024/03/06', [], ['informo_20240306.csv'])
('/home/code/inesdata-mov/data-generation/data/processed/informo/2024/03/07', [], ['informo_20240307.csv'])
('/home/code/inesdata-mov/data-generation/data/processed/informo/2024/03/12', [], ['informo_20240312.csv'])
('/home/code/inesdata-mov/data-generation/data/processed/informo/2024/03/02', [], ['informo_20240302.csv'])
('/home/code/inesdata-mov/data-generation/data/processed/informo/2024/03/05', [], ['informo_20240305.csv'])
('/home/code/inesdata-mov/data-generation/data/processed/informo/2024/03/11', [], ['informo_20240311.csv'])
('/home/code/inesdata-mov/data-generation/data

**Cada fila de este dataset representa el estado del tráfico de Madrid, en una determinada zona (medida por un dispositivo concreto), para una fecha y hora concretos.**

<div class="admonition warning">
<p class="admonition-title">-</p>
<p>
Vamos a analizar la calidad del dataset generado solamente para el día 13 de marzo, en el futuro dispondremos de más días.
</p>
</div>

In [4]:
df = pd.read_csv(
    os.path.join(INFORMO_DATA_PATH, "2024", "03", "13", "informo_20240313.csv"),
    parse_dates=["date", "datetime"],
)
df

,idelem,descripcion,accesoAsociado,intensidad,ocupacion,carga,nivelServicio,intensidadSat,error,subarea,st_x,st_y,velocidad,datetime,date
0,6711,NaN,NaN,3840,30,105,1.0,NaN,N,NaN,"442306,196584557","4481615,71687789",37.0,2024-03-13 07:55:08,2024-03-13
1,6039,Torrelaguna - Av. Ramón y Cajal-Acceso M30,4628002.0,840,6,24,0.0,3000.0,N,3215.0,"443984,139107713","4478277,30226478",NaN,2024-03-13 07:55:08,2024-03-13
2,6040,Torrelaguna - Sorzano-Acceso M30,4628001.0,520,5,31,1.0,2000.0,N,3215.0,"444079,304201131","4478026,60397703",NaN,2024-03-13 07:55:08,2024-03-13
3,6041,Av. Ramón y Cajal - Puente M30-Torrelaguna,4627003.0,1400,4,34,1.0,3380.0,N,3215.0,"443837,934167538","4478283,27889757",NaN,2024-03-13 07:55:08,2024-03-13
4,6042,José Silva - Agastia-Torrelaguna,4627004.0,600,6,24,0.0,2112.0,N,3203.0,"444106,480195963","4478409,94175933",NaN,2024-03-13 07:55:08,2024-03-13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
853109,10031,Narcís Monturiol N-S - Sangenjo-Av. Ilustración,4304002.0,20,0,2,0.0,775.0,N,314.0,"440656,756311705","4481793,23972478",NaN,2024-03-13 22:50:11,2024-03-13
853110,10463,Av. Ilustración O-E (Azobispo Morcillo - Nudo ...,NaN,700,1,14,0.0,4420.0,N,314.0,"440789,229442877","4481734,1912437",NaN,2024-03-13 22:50:11,2024-03-13
853111,3421,Bravo Murillo E-O - Pl.Castilla-Conde Serrallo,6201004.0,480,1,16,0.0,2900.0,N,304.0,"441453,970035479","4479675,62306307",NaN,2024-03-13 22:50:11,2024-03-13
853112,3423,Lateral Pº Castellana N-S - Pl.Castilla-Rosari...,6003012.0,280,2,8,0.0,3200.0,N,301.0,"441493,761559993","4479352,76508833",NaN,2024-03-13 22:50:11,2024-03-13


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 853114 entries, 0 to 853113
Data columns (total 15 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   idelem          853114 non-null  int64         
 1   descripcion     801383 non-null  object        
 2   accesoAsociado  714568 non-null  float64       
 3   intensidad      853114 non-null  int64         
 4   ocupacion       853114 non-null  int64         
 5   carga           853114 non-null  int64         
 6   nivelServicio   852760 non-null  float64       
 7   intensidadSat   801383 non-null  float64       
 8   error           851431 non-null  object        
 9   subarea         801383 non-null  float64       
 10  st_x            853114 non-null  object        
 11  st_y            853114 non-null  object        
 12  velocidad       51731 non-null   float64       
 13  datetime        853114 non-null  datetime64[ns]
 14  date            853114 non-null  dat

In [6]:
df.columns

Index(['idelem', 'descripcion', 'accesoAsociado', 'intensidad', 'ocupacion',
       'carga', 'nivelServicio', 'intensidadSat', 'error', 'subarea', 'st_x',
       'st_y', 'velocidad', 'datetime', 'date'],
      dtype='object')

## Conversiones de tipos

In [7]:
num_cols = list(df.select_dtypes(include=np.number).columns)
cat_cols = list(df.select_dtypes(include=["object"]).columns)
date_cols = list(df.select_dtypes(exclude=[np.number, "object"]).columns)

print(f"Numeric cols: {num_cols}")
print(f"Categoric cols: {cat_cols}")
print(f"Date cols: {date_cols}")

Numeric cols: ['idelem', 'accesoAsociado', 'intensidad', 'ocupacion', 'carga', 'nivelServicio', 'intensidadSat', 'subarea', 'velocidad']
Categoric cols: ['descripcion', 'error', 'st_x', 'st_y']
Date cols: ['datetime', 'date']


## QA checks ✅

### Unicidad
Como hemos comentado anteriormente, **cada fila de este dataset representa el estado del tráfico de Madrid, en una determinada zona (medida por un dispositivo concreto), para una fecha y hora concretos.** Por tanto, las claves primarias de este dataset se conformarán teniendo en cuenta dichos atributos:

In [8]:
df['idelem'].nunique()

4766

In [9]:
df['datetime'].nunique()

179

In [10]:
#  TODO

### Exactitud y Completitud

In [11]:
#  TODO

### Consistencia y Validez

In [12]:
#  TODO

## PROFILING 📑

In [13]:
profile = ProfileReport(
    df,
    title="🚦 INFORMO QA",
    dataset={
        "description": "INFORMO - Estado del tráfico",
        "url": "https://informo.madrid.es/informo/tmadrid/pm.xml",
    },
    variables={
        "descriptions": {
            "PK": "Identificador único (Primary Key) del dataset, compuesto por <datetime>_TODO",
            "date": "Fecha de la petición a la API",
            "datetime": "Fecha y hora de la petición a la API",
            "idelem": "Identificador del punto de medida. Permite su posicionamiento sobre plano e identificación del vial y sentido de la circulación",
            "descripcion": "Denominación del punto de medida",
            "accesoAsociado": "Código de control relacionado con el control semafórico para la modificación de los tiempos",
            "intensidad": "Intensidad de número de vehículos por hora. Un valor negativo implica la ausencia de datos",
            "ocupacion": "Porcentaje de tiempo que está un detector de tráfico ocupado por un vehículo",
            "carga": "Parámetro de carga del vial. Representa una estimación del grado",
            "nivelServicio": "Parámetro calculado en función de la velocidad y la ocupación",
            "intensidadSat": "Intensidad de saturación de la vía en veh/hora",
            "error": "Código de control de la validez de los datos del punto de medida",
            "subarea": "Identificador de la subárea de explotación de tráfico a la que pertenece el punto de medida",
            "st_x": "Coordenada X UTM del centroide que representa al punto de medida en el fichero georreferenciado",
            "st_y": "Coordenada Y UTM del centroide que representa al punto de medida en el fichero georreferenciado",
            "velocidad": "Velocidad medida",
        }
    },
    interactions=None,
    explorative=True,
    dark_mode=True,
)
profile.to_file(os.path.join(ROOT_PATH, "docs", "qa", "informo_report.html"))
# profile.to_notebook_iframe()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

## EDA 📊🔍

In [ ]:
#  TODO